# Система непересекающихся множеств


## Введение


**Определение:** Система непересекающихся множеств (Disjoint Set Union, DSU) - структура данных, которая поддерживает набор непересекающихся множеств и предоставляет операции для работы с ними.

**Определение:** Непересекающиеся множества - множества, не имеющие общих элементов: $\forall i \neq j: S_i \cap S_j = \emptyset$.

Система непересекающихся множеств поддерживает следующие операции:
1. **make_set(x)** - создает новое множество, содержащее только элемент x
2. **find(x)** - возвращает представителя множества, содержащего элемент x
3. **union(x, y)** - объединяет множества, содержащие элементы x и y

**Применения:**
1. Нахождение компонент связности в графе
2. Построение минимального остовного дерева (алгоритм Крускала)
3. Проверка эквивалентности в системах типов
4. Кластеризация данных


## Наивная реализация


**Идея:** Представим каждое множество в виде дерева, где каждый узел указывает на своего родителя. Корень дерева является представителем множества и указывает сам на себя.


In [ ]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
    
    def make_set(self, x):
        self.parent[x] = x
    
    def find(self, x):
        if self.parent[x] == x:
            return x
        return self.find(self.parent[x])
    
    def union(self, x, y):
        x_root = self.find(x)
        y_root = self.find(y)
        if x_root != y_root:
            self.parent[y_root] = x_root

# Пример использования
ds = DisjointSet(5)
for i in range(5):
    ds.make_set(i)

ds.union(0, 2)
ds.union(1, 3)
ds.union(0, 4)

print([ds.find(i) for i in range(5)])


**Теорема**
Асимптотика операций в наивной реализации DSU:
- make_set: $O(1)$
- find: $O(h)$, где $h$ - высота дерева
- union: $O(h)$

$\square$
Для make_set очевидно, что операция выполняется за константное время.

Для find мы проходим по цепочке родителей от узла до корня, что занимает время, пропорциональное высоте дерева.

Для union мы выполняем два поиска (find) и одну операцию присваивания, поэтому асимптотика также $O(h)$.
$\blacksquare$

**Замечание:** В худшем случае высота дерева может достигать $O(n)$, что делает операции find и union линейными по времени.


## Оптимизация: сжатие путей (Path Compression)


**Идея:** При выполнении операции find, мы можем "сжать путь", делая все узлы на пути от x до корня прямыми потомками корня. Это уменьшает высоту дерева и ускоряет последующие операции find.


In [ ]:
class DisjointSetWithPathCompression:
    def __init__(self, n):
        self.parent = list(range(n))
    
    def make_set(self, x):
        self.parent[x] = x
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        x_root = self.find(x)
        y_root = self.find(y)
        if x_root != y_root:
            self.parent[y_root] = x_root

# Пример использования
ds = DisjointSetWithPathCompression(5)
for i in range(5):
    ds.make_set(i)

ds.union(0, 2)
ds.union(1, 3)
ds.union(0, 4)

print([ds.find(i) for i in range(5)])


**Теорема**
Сжатие путей уменьшает высоту дерева и улучшает асимптотику операций.

$\square$
После выполнения find(x) с оптимизацией сжатия путей, все узлы на пути от x до корня становятся прямыми потомками корня. Это уменьшает высоту дерева и ускоряет последующие операции find для этих узлов.
$\blacksquare$


## Оптимизация: объединение по рангу (Union by Rank)


**Идея:** При выполнении операции union, мы можем выбирать, какое дерево присоединить к какому. Если присоединять более низкое дерево к более высокому, то высота результирующего дерева не увеличится, если ранги деревьев различны.

**Определение:** Ранг узла - это верхняя граница высоты поддерева с корнем в этом узле.


In [ ]:
class DisjointSetWithUnionByRank:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def make_set(self, x):
        self.parent[x] = x
        self.rank[x] = 0
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        x_root = self.find(x)
        y_root = self.find(y)
        
        if x_root == y_root:
            return
            
        if self.rank[x_root] < self.rank[y_root]:
            self.parent[x_root] = y_root
        else:
            self.parent[y_root] = x_root
            if self.rank[x_root] == self.rank[y_root]:
                self.rank[x_root] += 1

# Пример использования
ds = DisjointSetWithUnionByRank(5)
for i in range(5):
    ds.make_set(i)

ds.union(0, 2)
ds.union(1, 3)
ds.union(0, 4)

print([ds.find(i) for i in range(5)])
print(ds.rank)


**Теорема**
При использовании объединения по рангу, высота дерева с n узлами не превышает $\log_2 n$.

$\square$
Докажем индукцией по количеству операций union.

База: После выполнения make_set для всех элементов, высота каждого дерева равна 0, ранг каждого узла равен 0.

Индукционный переход: Пусть утверждение верно для первых k операций union. Рассмотрим (k+1)-ю операцию union(x, y).

Если ранги корней x_root и y_root различны, то высота результирующего дерева не изменится, так как мы присоединяем дерево с меньшим рангом к дереву с большим рангом.

Если ранги равны, то ранг корня результирующего дерева увеличится на 1. Пусть ранг корня был r. Тогда в каждом из двух объединяемых деревьев было не менее $2^r$ узлов (по индукционному предположению). После объединения в дереве будет не менее $2 \cdot 2^r = 2^{r+1}$ узлов, что соответствует новому рангу r+1.

Таким образом, для дерева с n узлами, ранг корня не превышает $\log_2 n$, а значит и высота дерева не превышает $\log_2 n$.
$\blacksquare$


## Амортизированный анализ


**Теорема**
При использовании как сжатия путей, так и объединения по рангу, амортизированная сложность операций make_set, find и union составляет $O(\alpha(n))$, где $\alpha(n)$ - обратная функция Аккермана, которая растет очень медленно и для всех практических значений n не превышает 4.

$\square$
Доказательство этой теоремы довольно сложное и основано на анализе потенциальной функции. Ключевая идея заключается в том, что после выполнения последовательности из m операций (где m ≥ n), общее время выполнения составляет $O(m \cdot \alpha(n))$.

Обратная функция Аккермана $\alpha(n)$ определяется как минимальное число k такое, что $A(k, \lfloor n/3 \rfloor) \geq \log_2 n$, где A - функция Аккермана.

Функция Аккермана растет чрезвычайно быстро, поэтому ее обратная функция растет очень медленно. Для всех практических значений n, $\alpha(n) \leq 4$.
$\blacksquare$

**Следствие**
Для практических применений, операции DSU с оптимизациями можно считать выполняющимися за почти константное время.


## Полная реализация DSU с оптимизациями


In [ ]:
class DisjointSetOptimized:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def make_set(self, x):
        self.parent[x] = x
        self.rank[x] = 0
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        x_root = self.find(x)
        y_root = self.find(y)
        
        if x_root == y_root:
            return
            
        if self.rank[x_root] < self.rank[y_root]:
            self.parent[x_root] = y_root
        else:
            self.parent[y_root] = x_root
            if self.rank[x_root] == self.rank[y_root]:
                self.rank[x_root] += 1

# Пример: нахождение компонент связности в графе
def find_connected_components(edges, n):
    dsu = DisjointSetOptimized(n)
    for i in range(n):
        dsu.make_set(i)
    
    for u, v in edges:
        dsu.union(u, v)
    
    components = {}
    for i in range(n):
        root = dsu.find(i)
        if root not in components:
            components[root] = []
        components[root].append(i)
    
    return list(components.values())

# Пример использования
edges = [(0, 1), (1, 2), (3, 4), (5, 6), (6, 7), (8, 9)]
n = 10
components = find_connected_components(edges, n)
print("Компоненты связности:")
for i, component in enumerate(components):
    print(f"Компонента {i+1}: {component}")


## Применение: алгоритм Крускала


**Определение:** Минимальное остовное дерево (Minimum Spanning Tree, MST) - это подграф взвешенного неориентированного графа, который является деревом и включает все вершины исходного графа, при этом сумма весов его ребер минимальна.

**Алгоритм Крускала:**
1. Отсортировать все ребра графа по возрастанию веса
2. Создать DSU с n элементами (по одному для каждой вершины)
3. Перебирать ребра в порядке возрастания веса:
   - Если концы ребра находятся в разных компонентах (разных множествах в DSU), добавить ребро в MST и объединить компоненты
   - Иначе пропустить ребро


In [ ]:
def kruskal_mst(edges, n):
    # Сортировка ребер по весу
    edges.sort(key=lambda x: x[2])
    
    dsu = DisjointSetOptimized(n)
    for i in range(n):
        dsu.make_set(i)
    
    mst = []
    total_weight = 0
    
    for u, v, weight in edges:
        if dsu.find(u) != dsu.find(v):
            dsu.union(u, v)
            mst.append((u, v, weight))
            total_weight += weight
    
    return mst, total_weight

# Пример использования
edges = [
    (0, 1, 10), (0, 2, 6), (0, 3, 5),
    (1, 3, 15), (2, 3, 4)
]
n = 4
mst, total_weight = kruskal_mst(edges, n)

print("Минимальное остовное дерево:")
for u, v, w in mst:
    print(f"Ребро ({u}, {v}) с весом {w}")
print(f"Общий вес MST: {total_weight}")


**Теорема**
Алгоритм Крускала находит минимальное остовное дерево взвешенного неориентированного графа.

$\square$
Докажем корректность алгоритма Крускала с помощью свойства разреза.

**Свойство разреза:** Пусть S - подмножество вершин графа, а (u, v) - ребро минимального веса, пересекающее разрез (S, V-S). Тогда существует минимальное остовное дерево, содержащее ребро (u, v).

Алгоритм Крускала на каждом шаге выбирает ребро минимального веса, соединяющее две различные компоненты связности (множества в DSU). Это ребро является ребром минимального веса, пересекающим разрез между этими компонентами. По свойству разреза, такое ребро должно входить в некоторое минимальное остовное дерево.

Таким образом, алгоритм Крускала на каждом шаге добавляет ребро, которое может быть частью минимального остовного дерева, и в итоге строит минимальное остовное дерево.
$\blacksquare$

**Теорема**
Временная сложность алгоритма Крускала составляет $O(E \log E)$, где E - количество ребер в графе.

$\square$
Сортировка ребер занимает $O(E \log E)$ времени.
Выполнение операций DSU для всех ребер занимает $O(E \cdot \alpha(V))$ времени, где $\alpha$ - обратная функция Аккермана.
Поскольку $\alpha(V)$ растет очень медленно и практически является константой, общая временная сложность определяется сортировкой и составляет $O(E \log E)$.
$\blacksquare$


## Заключение


Система непересекающихся множеств (DSU) - это эффективная структура данных для работы с непересекающимися множествами. С оптимизациями сжатия путей и объединения по рангу, DSU обеспечивает почти константное время выполнения операций, что делает ее чрезвычайно полезной в различных алгоритмах, особенно связанных с графами.

Основные применения DSU включают:
1. Нахождение компонент связности в графе
2. Построение минимального остовного дерева (алгоритм Крускала)
3. Обнаружение циклов в неориентированном графе
4. Решение задач о связности и эквивалентности

DSU является примером структуры данных, где простая идея в сочетании с несколькими оптимизациями приводит к чрезвычайно эффективному решению.